In [2]:
%load_ext autoreload
%autoreload 2

In [22]:
from argparse import Namespace
from pathlib import Path
import numpy as np
import scipy.io as sio 

import torch
import torch.nn.functional as F

from turboflow.dataloaders import TurboFlowDataModule

from turboflow.utils import phy_utils as phy
from turboflow.utils import torch_utils as tch

import matplotlib.pyplot as plt

In [16]:
# load dataset
data_dir = Path('/','home','dicarlo_d','Documents','Datasets','Turb2D.hdf5')
dm = TurboFlowDataModule(dataset='Turb2D', 
                         data_dir=data_dir,
                         batch_size=100000,
                         time_idx=666,
                         train_downsampling_space=1,
                         val_downsampling_space=1,
                         test_downsampling_space=1,
                         train_downsampling_time=1,
                         val_downsampling_time=1,
                         test_downsampling_time=1,
                         num_workers=1)
dm.setup()

In [17]:
X_lr, y_lr = dm.val_dataset[:]
img_shape_lr = dm.val_dataset.img_shape[:2]
shape_lr = dm.val_dataset.img_shape
L = shape_lr[0]
N_lr = X_lr.shape[0]
print('Low Resolution data:', N_lr, L)

X_hr, y_hr = dm.test_dataset[:]
img_shape_hr = dm.test_dataset.img_shape[:2]
shape_hr = dm.test_dataset.img_shape
H = shape_hr[0]
N_hr = X_hr.shape[0]
print('High Resolution data:', N_hr, H)

Low Resolution data: 65536 256
High Resolution data: 65536 256


In [25]:
coords_lr = X_lr.reshape(256,256,2).numpy()
vels_lr = y_lr.reshape(256,256,2).numpy()

mat = {}
mat['coords'] = coords_lr
mat['vels'] = vels_lr

sio.savemat('./turb2D_sample666_xy_uv_256x256x2.mat', mat)

In [ ]:
# compute and visualize divergence at highest scale
uv = y_hr.reshape(shape_hr)
xy = X_hr.reshape(shape_hr)
print(uv.shape)
print(type(uv))

x = xy[:,:,0].numpy()
y = xy[:,:,1].numpy()
u = uv[:,:,0].numpy()
v = uv[:,:,1].numpy()

plt.figure(figsize=(20,5))
plt.subplot(141)
plt.imshow(x)
plt.subplot(142)
plt.imshow(y)
plt.subplot(143)
plt.imshow(u)
plt.subplot(144)
plt.imshow(v)
plt.show()

D = phy.compute_divergence((x,y),(u,v))
D = D[2:-2,2:-2]
plt.imshow(D)
plt.colorbar()
plt.show()
print(np.linalg.norm(D))

W = phy.compute_vorticity((x,y),(u,v))
plt.imshow(W)
plt.colorbar()
plt.show()

In [ ]:
ds = 4
plt.figure(figsize=(10,10))
plt.quiver (x[::ds,::ds], y[::ds,::ds],
            u[::ds,::ds], v[::ds,::ds])
plt.show()

In [ ]:
R, C = u.shape
print(R, C)
padd = 2
u_pad = np.pad(u, (padd*R,padd*C), mode='symmetric')
v_pad = np.pad(v, (padd*R,padd*C), mode='symmetric')

print(u_pad.shape)
plt.imshow(u_pad)
plt.show()

nR, nC = u_pad.shape

In [ ]:
beta = np.arange(nR) - nR/2 + 0.5
t = np.ones(nC)
print(beta[0], beta[-1])
beta = beta[:,None]*t[None,:]

alpha = np.arange(nC) - nC/2 + 0.5
t = np.ones(nR)
print(alpha[0], alpha[-1])
alpha = alpha[None,:]*t[:,None]

plt.imshow(beta)
plt.show()

plt.imshow(alpha)
plt.show()

In [ ]:
coef = 2*1j*np.pi / np.sqrt(nR*nC)

In [ ]:
Fu = np.fft.fftshift( np.fft.fft2(u, s=(nR,nC)))
Fv = np.fft.fftshift( np.fft.fft2(v, s=(nR,nC)))

print(Fu.shape, Fv.shape)

Fs = (alpha * Fv - beta * Fu) / (coef*(alpha * alpha + beta * beta))
Fp = (alpha * Fu + beta * Fv) / (coef*(alpha * alpha + beta * beta))

print(Fs.shape, Fp.shape)

plt.imshow(np.abs(Fu))
plt.show()

plt.imshow(np.abs(Fv))
plt.show()


In [ ]:
sf_t = np.fft.ifft2(np.fft.ifftshift(Fs), s=(nR,nC))
fp_t = np.fft.ifft2(np.fft.ifftshift(Fp), s=(nR,nC))

print(sf_t.shape)
plt.imshow(np.abs(sf_t))
plt.show()

plt.imshow(np.real(sf_t))
plt.colorbar()
plt.show()

plt.imshow(np.imag(sf_t))
plt.colorbar()
plt.show()

print(fp_t.shape)
plt.imshow(np.abs(fp_t))
plt.show()

In [ ]:
R1 = nR // 2
C1 = nC // 2
R2 = R1 + R
C2 = C1 + C

R1, R2 = 0, 256
C1, C2 = 0, 256

print(R1, R2)
print(C1, C2)

plt.imshow(np.abs(sf_t[R1:R2,C1:C2]))
plt.show()

plt.imshow(np.abs(fp_t[R1:R2,C1:C2]))
plt.show()

In [ ]:
sf = sf_t[R1:R2,C1:C2]
fp = fp_t[R1:R2,C1:C2]

stf=np.real(sf)
fpt=np.real(fp)

plt.imshow(stf)
plt.show()

plt.imshow(fpt)
plt.show()

In [ ]:
print(x.shape)
print(fpt.shape)

uirr, virr = phy.my_grad([fpt], [x[:,0], y[0,:]], indexing='ij')[0]
vsol, usol = phy.my_grad([stf], [x[:,0], y[0,:]], indexing='ij')[0]
usol = -usol

plt.imshow(uirr)
plt.colorbar()
plt.show()
plt.imshow(virr)
plt.show()

ds = 4
plt.figure(figsize=(10,10))
plt.quiver(x[::ds,::ds], y[::ds,::ds],
          uirr[::ds,::ds], virr[::ds,::ds])
plt.show()

In [ ]:
plt.imshow(usol)
plt.show()
plt.imshow(vsol)
plt.show()

ds = 4
plt.figure(figsize=(10,10))
plt.quiver(x[::ds,::ds], y[::ds,::ds],
          usol[::ds,::ds], vsol[::ds,::ds])
plt.show()

In [ ]:
ds = 4
plt.figure(figsize=(10,10))
plt.quiver(x[::ds,::ds], y[::ds,::ds],
           u[::ds,::ds], v[::ds,::ds])
plt.show()

ds = 4
plt.figure(figsize=(10,10))
plt.quiver(x[::ds,::ds], y[::ds,::ds],
           (uirr + usol)[::ds,::ds], (virr+vsol)[::ds,::ds])
plt.show()

In [ ]:
ulam = u - (uirr + usol)
plt.imshow(ulam)